## <u>Introduction and Note</u>

In [1]:
!mkdir "trainer/aug_sample_images"

mkdir: cannot create directory ‘trainer/aug_sample_images’: File exists


In [2]:
# !mkdir validation_image_results

## <u>Prepare Data</u>

In [3]:
import shutil
import glob as glob
import random
import pandas as pd
import xml.etree.ElementTree as ET
import pandas as pd
import os

from tqdm import tqdm

In [4]:
"""
Copy the images and XML files from original folders to the temporary folders
"""


ROOT_PATH = "./" # YOU MAY NEED TO CHNAGE THIS
DATA_FOLDER = 'data'

#################################
# chage the path after {ROOT_PATH}/
os.makedirs(f"{ROOT_PATH}/{DATA_FOLDER}", exist_ok=True) 
os.makedirs(f"{ROOT_PATH}/{DATA_FOLDER}/train", exist_ok=True)
os.makedirs(f"{ROOT_PATH}/{DATA_FOLDER}/validation", exist_ok=True)
os.makedirs(f"{ROOT_PATH}/{DATA_FOLDER}/test", exist_ok=True)
os.makedirs(f"{ROOT_PATH}/{DATA_FOLDER}/train/images", exist_ok=True)
os.makedirs(f"{ROOT_PATH}/{DATA_FOLDER}/train/annotations", exist_ok=True)
os.makedirs(f"{ROOT_PATH}/{DATA_FOLDER}/validation/images", exist_ok=True)
os.makedirs(f"{ROOT_PATH}/{DATA_FOLDER}/validation/annotations", exist_ok=True)
os.makedirs(f"{ROOT_PATH}/{DATA_FOLDER}/test/images", exist_ok=True)
os.makedirs(f"{ROOT_PATH}/{DATA_FOLDER}/test/annotations", exist_ok=True)
os.makedirs(f"{ROOT_PATH}/{DATA_FOLDER}/train/labels", exist_ok=True)
os.makedirs(f"{ROOT_PATH}/{DATA_FOLDER}/validation/labels", exist_ok=True)
os.makedirs(f"{ROOT_PATH}/{DATA_FOLDER}/test/labels", exist_ok=True)

#################################
# make final training and validation folders
# MOST PROBABLY NO NEED TO CHNAGE THESE
os.makedirs(f"{ROOT_PATH}/final_train_images", exist_ok=True)
os.makedirs(f"{ROOT_PATH}/final_valid_images", exist_ok=True)
os.makedirs(f"{ROOT_PATH}/final_test_images", exist_ok=True)
os.makedirs(f"{ROOT_PATH}/final_labels", exist_ok=True)

In [10]:
COPY_FROM_ROOT_PATH = '../input/ships_dataset/ShipsPascalVOC'
ANNOT_PATH = f"{COPY_FROM_ROOT_PATH}/annotations"
IMAGE_PATH = f"{COPY_FROM_ROOT_PATH}/images"

COPY_TO_ROOT_PATH = 'data'

annotations = glob.glob(f"{ANNOT_PATH}/*.xml")
images = glob.glob(f"{IMAGE_PATH}/*.png")

annotations.sort()
images.sort()

image_names = []
annot_names = []

print(annotations[:1])
print(images[:1])

print(f"Total number of training images: {len(images)}")
print(f"Total number of training annotations: {len(annotations)}")

for image in images:
    image_name = image.split('.')[0]
    image_names.append(image_name)
for annot in annotations:
    annot_name = annot.split('.')[0]
    annot_names.append(annot_name)
    

# check for images without XML files
extra_image_counter = 0
for num, name in enumerate(image_names):
    if name not in annot_names:
        print(f"EXTRA IMAGE: {image_name}.png")
        extra_image_counter += 1
# check for XML files without images
extra_xml_counter = 0
for num, name in enumerate(annot_names):
    if name not in image_names:
        print(f"EXTRA XML: {annot_name}.xml")
        extra_xml_counter += 1
print(f"Total extra images: {extra_image_counter}")
print(f"Total extra XMLs: {extra_xml_counter}")

assert (len(annotations) == len(images)), 'Number of images and annotataion files do not match'

train_split = 80
valid_split = 10
test_split = 10

# set the train, evaluation/validation, and test ratio
train_ratio, valid_ratio, test_ratio = int((train_split/100)*len(images)), int((valid_split/100)*len(images)), int((test_split/100)*len(images))
print(f"Train data points: {train_ratio}")
print(f"Validation/evaluation data points: {valid_ratio}")
print(f"Test data points: {test_ratio}")

#############################################################
random.seed(42)
r = random.random() # randomly generating a real in [0,1)

# for training data
temp_annotations = glob.glob(f"{ANNOT_PATH}/*.xml")
temp_images = glob.glob(f"{IMAGE_PATH}/*.png")
temp_annotations.sort()
temp_images.sort()
random.shuffle(temp_annotations, lambda : r)  # lambda : r is an unary function which returns r
random.shuffle(temp_images, lambda : r) # using the same function as used in prev line so that shuffling order is same
for i in tqdm(range(0, train_ratio)):
    if i < 5:
        print(temp_annotations[i].split('.')[0].split(os.path.sep)[-1], temp_images[i].split('.')[0].split(os.path.sep)[-1])
    assert(temp_annotations[i].split('.')[0].split(os.path.sep)[-1] == temp_images[i].split('.')[0].split(os.path.sep)[-1]), 'Annotation and image do not match'
    shutil.copy(temp_annotations[i], f"{COPY_TO_ROOT_PATH}/train/annotations")
    shutil.copy(temp_images[i], f"{COPY_TO_ROOT_PATH}/train/images")

# for validation dataset
temp_annotations = glob.glob(f"{ANNOT_PATH}/*.xml")
temp_images = glob.glob(f"{IMAGE_PATH}/*.png")
temp_annotations.sort()
temp_images.sort()
random.shuffle(temp_annotations, lambda : r)  # lambda : r is an unary function which returns r
random.shuffle(temp_images, lambda : r) # using the same function as used in prev line so that shuffling order is same
for i in tqdm(range(train_ratio, train_ratio+valid_ratio)):
    if i < 5:
        print(temp_annotations[i].split('.')[0].split(os.path.sep)[-1], temp_images[i].split('.')[0].split(os.path.sep)[-1])
    assert(temp_annotations[i].split('.')[0].split(os.path.sep)[-1] == temp_images[i].split('.')[0].split(os.path.sep)[-1]), 'Annotation and image do not match'
    shutil.copy(temp_annotations[i], f"{COPY_TO_ROOT_PATH}/validation/annotations")
    shutil.copy(temp_images[i], f"{COPY_TO_ROOT_PATH}/validation/images")

# for test data
temp_annotations = glob.glob(f"{ANNOT_PATH}/*.xml")
temp_images = glob.glob(f"{IMAGE_PATH}/*.png")
temp_annotations.sort()
temp_images.sort()
random.shuffle(temp_annotations, lambda : r)  # lambda : r is an unary function which returns r
random.shuffle(temp_images, lambda : r) # using the same function as used in prev line so that shuffling order is same
for i in tqdm(range(train_ratio+valid_ratio, train_ratio+valid_ratio+test_ratio)):
    if i < 5:
        print(temp_annotations[i].split('.')[0].split(os.path.sep)[-1], temp_images[i].split('.')[0].split(os.path.sep)[-1])
    assert(temp_annotations[i].split('.')[0].split(os.path.sep)[-1] == temp_images[i].split('.')[0].split(os.path.sep)[-1]), 'Annotation and image do not match'
    shutil.copy(temp_annotations[i], f"{COPY_TO_ROOT_PATH}/test/annotations")
    shutil.copy(temp_images[i], f"{COPY_TO_ROOT_PATH}/test/images")

['../input/ships_dataset/ShipsPascalVOC/annotations/boat0.xml']
['../input/ships_dataset/ShipsPascalVOC/images/boat0.png']
Total number of training images: 621
Total number of training annotations: 621
Total extra images: 0
Total extra XMLs: 0
Train data points: 496
Validation/evaluation data points: 62
Test data points: 62


  0%|          | 2/496 [00:00<00:28, 17.51it/s]

100%|██████████| 62/62 [00:00<00:00, 416.89it/s]


In [11]:
def xml_to_csv(path):
    annotations_list = glob.glob(path + '/*.xml')
    annotations_list.sort()
    xml_list = []
    for xml_file in annotations_list:
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            # print('MEMBER', member)
            try:
                value = (root.find('filename').text,
                         int(root.find('size')[0].text),
                         int(root.find('size')[1].text),
                         member[0].text,
                         int(member[4][0].text),
                         int(member[4][1].text),
                         int(member[4][2].text),
                         int(member[4][3].text)
                         )
            except:
                value = (root.find('filename').text,
                         int(root.find('size')[0].text),
                         int(root.find('size')[1].text),
                         member[0].text,
                         int(member[5][0].text),
                         int(member[5][1].text),
                         int(member[5][2].text),
                         int(member[5][3].text)
                         )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


def main():
    ROOT_PATH = 'data'
    for folder in [
        f"{ROOT_PATH}/train", 
        f"{ROOT_PATH}/validation",
        f"{ROOT_PATH}/test"]:
        xml_df = xml_to_csv(folder+'/'+'annotations')
        xml_df.to_csv((folder + '/' + 'labels/' + 'labels.csv'), index=None)
        print('Successfully converted xml to csv.')

main()

Successfully converted xml to csv.
Successfully converted xml to csv.
Successfully converted xml to csv.


In [15]:
ROOT_PATH = 'data'

train_csv = pd.read_csv(f"{ROOT_PATH}/train/labels/labels.csv")
final_train_csv = pd.concat([train_csv])
final_train_csv.to_csv(f"final_labels/final_train_csv.csv", index=False)

valid_csv = pd.read_csv(f"{ROOT_PATH}/validation/labels/labels.csv")
final_valid_csv = pd.concat([valid_csv])
final_valid_csv.to_csv(f"final_labels/final_valid_csv.csv", index=False)

test_csv = pd.read_csv(f"{ROOT_PATH}/test/labels/labels.csv")
final_test_csv = pd.concat([test_csv])
final_test_csv.to_csv(f"final_labels/final_test_csv.csv", index=False)

In [16]:
ROOT_PATH = 'data'

# for training images
train_images = glob.glob(f"{ROOT_PATH}/train/images/*.png")

for i in tqdm(range(len(train_images))):
    shutil.copy(train_images[i], f"final_train_images")

#######################
#######################
# for validation images

valid_images = glob.glob(f"{ROOT_PATH}/validation/images/*.png")

for i in tqdm(range(len(valid_images))):
    shutil.copy(valid_images[i], f"final_valid_images")

#######################
#######################
# for test images
test_images = glob.glob(f"{ROOT_PATH}/test/images/*.png")

for i in tqdm(range(len(test_images))):
    shutil.copy(test_images[i], f"final_test_images")

100%|██████████| 62/62 [00:00<00:00, 778.50it/s]


In [21]:
"""
Sometimes the xmin, ymin, xmax, and ymax annotation values may be 
larger than the actual image dimension due to labelling error.
This might cause unkown errors down the line. This Python script
removes such erroneous rows from the CSV files and rewrites them 
to the disk.
"""

# ROOT_PATH = '/content'

print('INFO: In test CSV')
test_df = pd.read_csv(f"final_labels/final_test_csv.csv")
drop_indices = []

for i in range(len(test_df)):
    w, h = test_df['width'][i], test_df['height'][i]
    if test_df['xmin'][i] > w or test_df['xmax'][i] > w or \
        test_df['ymin'][i] > h or test_df['ymax'][i] > h:
        print(test_df['filename'][i])
        drop_indices.append(test_df.index[i])
print(f"Drop indices: {drop_indices}")

test_df.drop(drop_indices, inplace=True)
# REWRITE THE CSV FILE TO DISK. VERY IMPORTANT
test_df.to_csv(f"final_labels/final_test_csv.csv", index=False)

##########################################################################

print('INFO: In valid CSV')
valid_df = pd.read_csv(f"final_labels/final_valid_csv.csv")
drop_indices = []

for i in range(len(valid_df)):
    w, h = valid_df['width'][i], valid_df['height'][i]
    if valid_df['xmin'][i] > w or valid_df['xmax'][i] > w \
        or valid_df['ymin'][i] > h or valid_df['ymax'][i] > h:
        print(valid_df['filename'][i])
        drop_indices.append(valid_df.index[i])
print(f"Drop indices: {drop_indices}")

valid_df.drop(drop_indices, inplace=True)
# REWRITE THE CSV FILE TO DISK. VERY IMPORTANT
valid_df.to_csv(f"final_labels/final_valid_csv.csv", index=False)
##########################################################################

print('INFO: In training CSV')
train_df = pd.read_csv(f"final_labels/final_train_csv.csv")
drop_indices = []

for i in range(len(train_df)):
    w, h = train_df['width'][i], train_df['height'][i]
    if train_df['xmin'][i] > w or train_df['xmax'][i] > w \
        or train_df['ymin'][i] > h or train_df['ymax'][i] > h:
        print(train_df['filename'][i])
        drop_indices.append(train_df.index[i])
print(f"Drop indices: {drop_indices}")

train_df.drop(drop_indices, inplace=True)
# REWRITE THE CSV FILE TO DISK. VERY IMPORTANT
train_df.to_csv(f"final_labels/final_train_csv.csv", index=False)
##########################################################################

INFO: In test CSV
Drop indices: []
INFO: In valid CSV
Drop indices: []
INFO: In training CSV
Drop indices: []


In [22]:
train_df = pd.read_csv('final_labels/final_train_csv.csv')
valid_df = pd.read_csv('final_labels/final_valid_csv.csv')
test_df = pd.read_csv('final_labels/final_test_csv.csv')

print(f"Total number of final train images: {len(train_df.groupby(by='filename').count())}")
print(f"Total number of final validation images: {len(valid_df.groupby(by='filename').count())}")
print(f"Total number of final test/evaluation images: {len(test_df.groupby(by='filename').count())}")

print(f"CLASSES: {train_df['class'].unique()} \nTOTAL: {len(train_df['class'].unique())} classes")
print(f"CLASSES: {valid_df['class'].unique()}")
print(f"CLASSES: {test_df['class'].unique()}")

Total number of final train images: 496
Total number of final validation images: 62
Total number of final test/evaluation images: 62
CLASSES: ['boat'] 
TOTAL: 1 classes
CLASSES: ['boat']
CLASSES: ['boat']
